# Creazione del dataset con PRAW

## Collegamento al profilo

In [1]:
import praw
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import csv
import igraph as ig
import sys
sys.path.append('..')

In [2]:
reddit = praw.Reddit(
    client_id=os.getenv('reddit_client_id'),
    client_secret=os.getenv('reddit_client_secret'),
    user_agent=os.getenv('reddit_user_agent'),
    password=os.getenv('reddit_password'),
    username=os.getenv('reddit_username'))

## reddit_comments

In [3]:
def extract_comments(comment,post_id):
    comment_id = comment.id
    comment_body = comment.body
    comment_score = comment.score
    comment_replies = len(comment.replies)
    created_utc = comment.created_utc
    comment_author_name = comment.author.name if comment.author else "deleted"
    comment_author_id = comment.author.id if comment.author else "deleted"
    parent_author = comment.parent()
    if parent_author.id == post_id:
        parent_author_name = "OP"
        parent_author_id = "0"
    else:
        parent_author_name = parent_author.author.name if parent_author and parent_author.author else "deleted"
        parent_author_id = parent_author.author.id if parent_author and parent_author.author else "deleted"
    return comment_id, comment_body, comment_score, comment_replies, created_utc, comment_author_id, comment_author_name, parent_author_id, parent_author_name

In [14]:
def x(post_ids):
    post_to_download = []
    df = pd.read_csv('../reddit_comments.csv')
    df_values = df['post_id'].values
    for post_id in post_ids:
        if post_id not in df_values:
            post_to_download.append(post_id)
    return post_to_download

post_ids = ['17z2hci', '1bulhj9', '1b6cg4q', '17lese9', '1cwqkqe', '10v8sey', '197vo6o', '1d5h5h6','19aeo2k']

first_len = len(post_ids)

fieldnames = [
    'comment_id', 
    'comment_author_id', 
    'comment_author_name', 
    'comment_parent_id', 
    'comment_parent_name', 
    'comment_score', 
    'comment_replies', 
    'comment_posted_time', 
    'comment_body',
    'post_id',
    'post_title',
    'post_url',
    'post_subreddit'
]

post_ids = x(post_ids)
second_len = len(post_ids)

if second_len > first_len:
    mode = 'w'
else:
    mode = 'a'


# Open the file in append mode
with open('../reddit_comments.csv', mode, newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_MINIMAL)
    if mode == 'w':
        writer.writeheader()

    for post_id in post_ids:
        rows = []
        i = 1
        submission = reddit.submission(post_id)
        submission.comments.replace_more(limit=30)
        post_title = submission.title
        post_id = submission.id
        post_url = submission.url
        post_subreddit = submission.subreddit.name
        comments = submission.comments.list()
        length = len(comments)
        print(f"Processing post {post_title}\n", end='\r')
        for comment in comments:
            print(f"Processed {(i/length*100):.2f}% of comments", end='\r')
            try:
                comment_id, comment_body, comment_score, comment_replies, created_utc, comment_author_id, comment_author_name, parent_author_id, parent_author_name = extract_comments(comment,post_id)
                row = {
                    'comment_id' : comment_id, 
                    'comment_author_id' : comment_author_id,
                    'comment_author_name' : comment_author_name,
                    'comment_parent_id' : parent_author_id,
                    'comment_parent_name' : parent_author_name,
                    'comment_score' : comment_score,
                    'comment_replies' : comment_replies,
                    'comment_posted_time' : created_utc,
                    'comment_body'  : comment_body,
                    'post_id' : post_id,
                    'post_title' : post_title,
                    'post_url' : post_url,
                    'post_subreddit' : post_subreddit
                }
                rows.append(row)
            except Exception as e:
                pass
            i += 1
        writer.writerows(rows)

In [6]:
open_file = pd.read_csv('../reddit_comments.csv')

In [7]:
open_file

,comment_id,comment_author_id,comment_author_name,comment_parent_id,comment_parent_name,comment_score,comment_replies,comment_posted_time,comment_body,post_id,post_title,post_url,post_subreddit
0,k9yfohd,209w6fpg,moonwave91,0,OP,160,2,1.700439e+09,Il vero problema è l'immensa orda di giornalis...,17z2hci,Ho visto troppi post sulla Giulia Cecchettin,https://www.reddit.com/r/Italia/comments/17z2h...,t5_2rbm5
1,k9xjbzd,deleted,deleted,0,OP,310,3,1.700426e+09,"Non è un idiota l'ex ragazzo, è un assassino, ...",17z2hci,Ho visto troppi post sulla Giulia Cecchettin,https://www.reddit.com/r/Italia/comments/17z2h...,t5_2rbm5
2,k9zies2,e5pmjzs5c,MainDelay9804,0,OP,22,0,1.700459e+09,Sono sti ragazzetti sui social che ormai han p...,17z2hci,Ho visto troppi post sulla Giulia Cecchettin,https://www.reddit.com/r/Italia/comments/17z2h...,t5_2rbm5
3,k9xb0xr,utwr8mto,AlessandroIT,0,OP,83,1,1.700422e+09,"Non fatevi influenzare dai post, continuate ad...",17z2hci,Ho visto troppi post sulla Giulia Cecchettin,https://www.reddit.com/r/Italia/comments/17z2h...,t5_2rbm5
4,k9xmjhd,49phg2z0,CthulhuParty,0,OP,87,1,1.700427e+09,ti da fastidio perchè percepisci giustamente c...,17z2hci,Ho visto troppi post sulla Giulia Cecchettin,https://www.reddit.com/r/Italia/comments/17z2h...,t5_2rbm5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4032,kuou834,deleted,deleted,52vwr29q,srbin97,1,1,1.710344e+09,[deleted],19aeo2k,Eccovi il mio gatto,https://i.redd.it/qya4zxo41ddc1.jpeg,t5_2rbm5
4033,kup7o5m,deleted,deleted,deleted,deleted,0,2,1.710348e+09,[removed],19aeo2k,Eccovi il mio gatto,https://i.redd.it/qya4zxo41ddc1.jpeg,t5_2rbm5
4034,kupcj5c,deleted,deleted,deleted,deleted,1,1,1.710350e+09,[deleted],19aeo2k,Eccovi il mio gatto,https://i.redd.it/qya4zxo41ddc1.jpeg,t5_2rbm5
4035,kuqovvy,roytbcgg,Italia-ModTeam,deleted,deleted,1,0,1.710366e+09,###Rispetto reciproco\n\nNon sono permessi ins...,19aeo2k,Eccovi il mio gatto,https://i.redd.it/qya4zxo41ddc1.jpeg,t5_2rbm5
